# Fine-Tuning a Model

Let's teach our text to image generator what my dog looks like.

![redhat dog](https://rhods-public.s3.amazonaws.com/sample-data/images/redhat-dog-small.jpg)

### Check GPU

In [ ]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

## Install Requirements

In [ ]:
!wget https://rhods-public.s3.amazonaws.com/src/xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl -O xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl
!pip install -q xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl

In [ ]:
!pip install -q --upgrade accelerate transformers ftfy
!pip install -q --upgrade git+https://github.com/ShivamShrirao/diffusers
!pip install -q natsort bitsandbytes

In [ ]:
!pip list | grep -e torch -e torchvision -e diffusers -e accelerate -e torchvision -e transformers -e ftfy -e tensorboard -e Jinja2 -e xformers


In [ ]:
libcudart_files = !find / -name "libcudart.so*" 2> /dev/null
libcudart_files

In [ ]:
import os

# workaround https://github.com/TimDettmers/bitsandbytes/issues/62

libcudart_so_files = !find / -name "libcudart.so" 2> /dev/null
all_libcudart_files = !find / -name "libcudart.so*" 2> /dev/null

if len(libcudart_so_files) == 0:
    directory = os.path.dirname(all_libcudart_files[0])
    os.environ['LD_LIBRARY_PATH'] = directory
    !ln -s "${LD_LIBRARY_PATH}/libcudart.so.11.0" "${LD_LIBRARY_PATH}/libcudart.so" > /dev/null
    !find / -name "libcudart.so" 2> /dev/null
else:
    directory = os.path.dirname(libcudart_so_files[0])
    os.environ['LD_LIBRARY_PATH'] = directory


In [ ]:
!mkdir -p ~/.huggingface
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

## Training Settings

In [ ]:
import os
from datetime import datetime

date = datetime.now()
date_string = date.strftime("%Y%m%d%H%M%S")

VERSION = os.environ.get("VERSION", "custom")
MODEL_NAME = os.environ.get("MODEL_NAME", "runwayml/stable-diffusion-v1-5")
OUTPUT_DIR = os.path.join(os.getcwd(), f"{VERSION}/weights/redhat-dog")
DATA_DIR = os.path.join(os.getcwd(), f"{VERSION}/data")
INSTANCE_DATA_URL = os.environ.get("INSTANCE_DATA_URL", "https://rhods-public.s3.amazonaws.com/sample-data/images/redhat-dog.tar.gz")
INSTANCE_DIR = os.path.join(DATA_DIR, "instance_dir")
CLASS_DIR = os.path.join(DATA_DIR, "class_dir")
INSTANCE_PROMPT = os.environ.get("INSTANCE_PROMPT", "photo of a rhteddy dog")
CLASS_PROMPT = os.environ.get("CLASS_PROMPT", "a photo of dog")

NUM_CLASS_IMAGES = int(os.environ.get("NUM_CLASS_IMAGES", "200"))
MAX_TRAIN_STEPS = int(os.environ.get("MAX_TRAIN_STEPS", "800"))

ONNX_OUTPUT_DIR = os.path.join(os.getcwd(), f"{VERSION}/onnx-redhat-dog")

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(INSTANCE_DIR, exist_ok=True)

print(f"Weights will be saved at {OUTPUT_DIR}")
print(f"MODEL_NAME={MODEL_NAME}")
print(f"Training data located in {INSTANCE_DIR}")
print(f"Instance Prompt = {INSTANCE_PROMPT}")

## Fine Tuning

In [ ]:
concepts_list = [
    {
        "instance_prompt":      INSTANCE_PROMPT,
        "class_prompt":         CLASS_PROMPT,
        "instance_data_dir":    INSTANCE_DIR,
        "class_data_dir":       CLASS_DIR
    },
]

import json
import os
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

### Get Training Data


In [ ]:
import sys
import os
import tarfile
import urllib

url = INSTANCE_DATA_URL
output = f"instance-images.tar.gz"
urllib.request.urlretrieve(url, output)

!tar -xzf instance-images.tar.gz --strip-components=1 -C $INSTANCE_DIR

### Training Job Setup

In [ ]:
!wget -O train_dreambooth_shivamshrirao.py https://raw.githubusercontent.com/ShivamShrirao/diffusers/main/examples/dreambooth/train_dreambooth.py

In [ ]:
!echo "MODEL_NAME=$MODEL_NAME"
!echo "OUTPUT_DIR=$OUTPUT_DIR"
!echo "DATA_DIR=$DATA_DIR"
!echo "INSTANCE_DIR=$INSTANCE_DIR"
!echo "CLASS_DATA_URL=$CLASS_DATA_URL"
!echo "CLASS_DIR=$CLASS_DIR"
!echo "INSTANCE_PROMPT=$INSTANCE_PROMPT"
!echo "CLASS_PROMPT=$CLASS_PROMPT"
!echo "NUM_CLASS_IMAGES=$NUM_CLASS_IMAGES"
!echo "MAX_TRAIN_STEPS=$MAX_TRAIN_STEPS"

### Run the Fine Tuning Job

In [ ]:
!python3 train_dreambooth_shivamshrirao.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=200 \
  --sample_batch_size=4 \
  --max_train_steps=800 \
  --save_interval=10000 \
  --save_sample_prompt="$INSTANCE_PROMPT" \
  --concepts_list="concepts_list.json" 


In [ ]:
from natsort import natsorted
from glob import glob
import os

WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] OUTPUT_DIR={OUTPUT_DIR}")
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")


### View Sample Inference

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


weights_folder = OUTPUT_DIR
folders = sorted([f for f in os.listdir(weights_folder) if f != "0" and not f.startswith('.')], key=lambda x: int(x))

row = len(folders)
col = len(os.listdir(os.path.join(weights_folder, folders[0], "samples")))
scale = 4
fig, axes = plt.subplots(row, col, figsize=(col*scale, row*scale), gridspec_kw={'hspace': 0, 'wspace': 0})

for i, folder in enumerate(folders):
    folder_path = os.path.join(weights_folder, folder)
    image_folder = os.path.join(folder_path, "samples")
    images = [f for f in os.listdir(image_folder)]
    for j, image in enumerate(images):
        if row == 1:
            currAxes = axes[j]
        else:
            currAxes = axes[i, j]
        if i == 0:
            currAxes.set_title(f"Image {j}")
        if j == 0:
            currAxes.text(-0.1, 0.5, folder, rotation=0, va='center', ha='center', transform=currAxes.transAxes)
        image_path = os.path.join(image_folder, image)
        img = mpimg.imread(image_path)
        currAxes.imshow(img, cmap='gray')
        currAxes.axis('off')

plt.tight_layout()
plt.savefig('grid.png', dpi=72)

# Export to ONNX

In [ ]:
!pip install -q --upgrade accelerate transformers ftfy natsort
!pip install -q git+https://github.com/huggingface/diffusers
!pip install -q optimum["onnxruntime"]
!pip install -q optimum["onnxruntime-gpu"]
# !wget https://rhods-public.s3.amazonaws.com/src/xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl -O xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl
# !pip install xformers-0.0.21.dev584-cp39-cp39-manylinux2014_x86_64.whl

In [ ]:
from natsort import natsorted
from glob import glob
import os

WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] OUTPUT_DIR={OUTPUT_DIR}")
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")


In [ ]:
from optimum.onnxruntime import ORTStableDiffusionPipeline
import torch

model_id = WEIGHTS_DIR
pipeline = ORTStableDiffusionPipeline.from_pretrained(model_id, export=True)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# pipeline.to(device)
prompt = "photo of a rhteddy dog"
image = pipeline(prompt).images[0]
pipeline.save_pretrained(ONNX_OUTPUT_DIR)
image

### Did it look like Teddy?

## Save to S3

In [ ]:
import os
import boto3
import botocore

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
endpoint_url = os.environ.get('AWS_S3_ENDPOINT')
region_name = os.environ.get('AWS_DEFAULT_REGION')
bucket_name = os.environ.get('AWS_S3_BUCKET')

session = boto3.session.Session(aws_access_key_id=aws_access_key_id,
                                aws_secret_access_key=aws_secret_access_key)

s3_resource = session.resource(
    's3',
    config=botocore.client.Config(signature_version='s3v4'),
    endpoint_url=endpoint_url,
    region_name=region_name)

bucket = s3_resource.Bucket(bucket_name)


def upload_directory_to_s3(local_directory, s3_prefix):
    for root, dirs, files in os.walk(local_directory):
        for filename in files:
            file_path = os.path.join(root, filename)
            relative_path = os.path.relpath(file_path, local_directory)
            s3_key = os.path.join(s3_prefix, relative_path)
            print(f"{file_path} -> {s3_key}")
            bucket.upload_file(file_path, s3_key)
            
def list_all_objects():
    for obj in bucket.objects.all():
        print(obj.key, obj.last_modified)

upload_directory_to_s3(ONNX_OUTPUT_DIR, f"text-to-image/{VERSION}/onnx-redhat-dog")
list_all_objects()